In [1]:
from typing import List, Tuple, Optional
Scanner = List[List[int]]

In [2]:
def read_data(path) -> List[Scanner]:
    f = open(path,"r")
    lines = [line.strip() for line in f.readlines()]
    scanners = []
    current_scanner = []
    for line in lines:
        if len(line) == 0:
            scanners.append(current_scanner)
            current_scanner = []
        elif line.startswith("---"):
            continue
        else:
            point = [int(c) for c in line.split(",")]
            current_scanner.append(point)
    if len(current_scanner)>0:
        scanners.append(current_scanner)
    return scanners

In [3]:
demo_dataset = read_data("demo.txt")
full_dataset = read_data("data.txt")
assert len(demo_dataset) == 5
assert len(full_dataset) == 39
demo_dataset[0]

[[404, -588, -901],
 [528, -643, 409],
 [-838, 591, 734],
 [390, -675, -793],
 [-537, -823, -458],
 [-485, -357, 347],
 [-345, -311, 381],
 [-661, -816, -575],
 [-876, 649, 763],
 [-618, -824, -621],
 [553, 345, -567],
 [474, 580, 667],
 [-447, -329, 318],
 [-584, 868, -557],
 [544, -627, -890],
 [564, 392, -477],
 [455, 729, 728],
 [-892, 524, 684],
 [-689, 845, -530],
 [423, -701, 434],
 [7, -33, -71],
 [630, 319, -379],
 [443, 580, 662],
 [-789, 900, -551],
 [459, -707, 401]]

# Part 1

In [4]:
from itertools import permutations

def get_rotated(scanner: Scanner) -> List[Scanner]:
    """Get the 48 rotations of a scanner. Not sure why it's not 24."""
    scanners = []
    for dx in [-1, 1]:
        for dy in [-1, 1]:
            for dz in [-1, 1]:
                for p in permutations([0, 1, 2]):
                    new_scanner = []
                    for point in scanner:
                        x = dx * point[0]
                        y = dy * point[1]
                        z = dz * point[2]
                        temp = [x,y,z]
                        new_point = [temp[p[0]], temp[p[1]], temp[p[2]]]
                        new_scanner.append(new_point)
                    scanners.append(new_scanner)
    scanners.reverse()
    return scanners

In [5]:
def do_translate(p: List[int], translation: Tuple[int, int, int]) -> List[int]:
    return [p[0]+translation[0], p[1] + translation[1], p[2] + translation[2]]

In [6]:
def translate(points: Scanner, translation: Tuple[int, int, int]) -> Scanner:
    return [ do_translate(p, points) for p in points]

In [7]:
def merge(scan1: Scanner, scan2: Scanner, translation: Tuple[int, int, int]) -> Scanner:
    """Merge 2 scanner together"""
    new_scanner = [ p for p in scan1 ]
    for p in [do_translate(p, translation) for p in scan2]:
        if not p in new_scanner:
            new_scanner.append(p)
    return new_scanner

In [8]:
assert merge([[100, 100, 100]], [[200, 200, 200], [200, -100, 100]], (-100, 200, 0)) == [[100, 100, 100], [100, 400, 200]]

In [9]:
Matrix = List[List[List[int]]]
FastMatrix = List[List[int]]

In [10]:
def get_matrix(points: Scanner) -> Matrix:
    return [
        [ 
            [ p1[0] - p2[0], p1[1] - p2[1], p1[2] - p2[2]] for p2 in points
        ] for p1 in points
    ]

In [11]:
get_matrix([[0, 0, 0], [1, 2, 3], [4, 5, 6]])

[[[0, 0, 0], [-1, -2, -3], [-4, -5, -6]],
 [[1, 2, 3], [0, 0, 0], [-3, -3, -3]],
 [[4, 5, 6], [3, 3, 3], [0, 0, 0]]]

In [13]:
def get_fast_matrix(matrix: Matrix) -> FastMatrix:
    """Fast matrix are faster to check as we need to compare only a int instead of a tuple"""
    return [[sum(p) for p in row ] for row in matrix]

In [14]:
get_fast_matrix(get_matrix([[0, 0, 0], [1, 2, 3], [4, 5, 6]]))

[[0, -6, -15], [6, 0, -9], [15, 9, 0]]

In [15]:
def count_intersection(points1, points2) -> int:
    return len([p for p in points2 if p in points1])

In [16]:
def fast_count_intersection(points1, points2) -> int:
    """
    Computing the number of intersection is very costy.
    
    Most of the computing time is spend here.
    """
    return len([p for p in points2 if p in points1])

In [17]:
def get_translation(matrix1: Matrix, fmatrix1: FastMatrix, scan1: Scanner, matrix2: Matrix, fmatrix2: FastMatrix, scan2: Scanner) -> Tuple[int, int, int]:
    """Get the translation from the first scanner to the second or None if incompatible"""
    for i1 in range(len(matrix1)):
        for i2 in range(len(matrix2)):
            fast_count = fast_count_intersection(fmatrix1[i1], fmatrix2[i2])
            if fast_count >= 12:
                real_count = count_intersection(matrix1[i1], matrix2[i2])
                if real_count >= 12:
                    p1 = scan1[i1]
                    p2 = scan2[i2]
                    return (p1[0]-p2[0], p1[1]-p2[1], p1[2]-p2[2])
    return None

In [18]:
def get_translation_with_rotation(scan1: Scanner, scan2: Scanner, matrix1: Optional[Matrix] = None) -> Tuple[Tuple[int, int, int], Scanner]:
    matrix1 = matrix1 if matrix1 is not None else get_matrix(scan1)
    fmatrix1 = get_fast_matrix(matrix1)
    rotated = get_rotated(scan2)
    for rot in rotated:
        matrix2 = get_matrix(rot)
        fmatrix2 = get_fast_matrix(matrix2)
        translation = get_translation(matrix1, fmatrix1, scan1, matrix2, fmatrix2, rot)
        if translation is not None:
            return (translation, rot)
    return (None, None)
    

In [19]:
assert get_translation_with_rotation(demo_dataset[0], demo_dataset[1])[0] == (68,-1246,-43)

In [20]:
LIMIT = 10000

def count_beacon(dataset) -> int:
    todo = [scanner for scanner in dataset]
    current_map = todo.pop(0)
    current_matrix = get_matrix(current_map)
    count = 0
    while (len(todo) > 0) and count < LIMIT:
        count += 1
        scan = todo.pop(0)
        (translation, rot) = get_translation_with_rotation(current_map, scan, matrix1=current_matrix)
        if translation is not None:
            current_map = merge(current_map, rot, translation)
            current_matrix = get_matrix(current_map)
        else:
            # We might find it later
            todo.append(scan)
        # print("TODO:", len(todo))
    return len(current_map)
        

In [21]:
assert count_beacon(demo_dataset) == 79

In [26]:
%%time
count_beacon(full_dataset)

CPU times: user 21min 24s, sys: 152 ms, total: 21min 24s
Wall time: 21min 24s


467

# Part 2

In [22]:
def get_centres(dataset) -> int:
    todo = [scanner for scanner in dataset]
    current_map = todo.pop(0)
    current_matrix = get_matrix(current_map)
    centres = [(0,0,0)]
    while (len(todo) > 0):
        scan = todo.pop(0)
        (translation, rot) = get_translation_with_rotation(current_map, scan, matrix1=current_matrix)
        if translation is not None:
            current_map = merge(current_map, rot, translation)
            current_matrix = get_matrix(current_map)
            centres.append(translation)
        else:
            # We might find it later
            todo.append(scan)
        # print("TODO:", len(todo))
    return centres
        

In [23]:
def dist(p1, p2) -> int:
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1]) + abs(p1[2] - p2[2])

In [24]:
def largest_manhattan(dataset) -> int:
    centres = get_centres(dataset)
    distances = [ dist(centres[i], centres[j]) for i in range(len(centres)) for j in range(len(centres)) ]
    return max(distances)

In [25]:
assert largest_manhattan(demo_dataset) == 3621

In [27]:
%%time
largest_manhattan(full_dataset)

CPU times: user 21min 22s, sys: 520 ms, total: 21min 23s
Wall time: 21min 23s


12226